In [42]:
import pandas as pd
from loguru import logger
from pathlib import Path
from tqdm.notebook import tqdm
import os
import re
import sys
import json
from mri_data import utils
import subprocess
import nibabel as nib
import numpy as np
from reload_recursive import reload_recursive
# from nilearn import plotting

sys.path.insert(0, "/home/srs-9/Projects/ms_mri/analysis/thalamus/helpers")
import utils

In [43]:
reload_recursive(utils)
import utils

In [44]:
with open("/home/srs-9/Projects/ms_mri/data/subject-sessions-longit.json", 'r') as f:
    subject_sessions = json.load(f)

drive_root = Path("/media/smbshare")
# drive_root = Path("/mnt/h")

dataroot = drive_root / "srs-9/thalamus_project/data"
qc_root = drive_root / "srs-9/thalamus_project/qc"
all_subjects = [int(subid) for subid in list(subject_sessions.keys())]

screenshot_script = "/home/srs-9/Projects/ms_mri/analysis/thalamus/quality_control/save_screenshot.sh"
image_root = Path("/home/srs-9/Projects/ms_mri/analysis/thalamus/quality_control/images")

segmentations = [
    "choroid.nii.gz",
    "aseg-ventricles.nii.gz",
    "aseg-third_ventricle.nii.gz",
    "aseg-fourth_ventricle.nii.gz",
    "aseg-CSF.nii.gz",
    "peripheral_CSF_CHECK.nii.gz",
    "all_CSF.nii.gz"
]
all_subjects = [int(subid) for subid in list(subject_sessions.keys())]

subject_roots = {
    int(sub): dataroot / f"sub{sub}-{sessions[0]}" for sub, sessions in subject_sessions.items()
}

thalamic_nuclei = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12]


### Functions

In [45]:
def refresh_permissions(path):
    """Force WSL to refresh NTFS metadata cache"""
    # Touch the directory to invalidate cache
    subprocess.run(['touch', '-c', path], check=False)

def is_greater_than_zero(image):
    return (image > 0)

def is_equal_to(image, check):
    return np.isin(image, check)


def get_ref_slice(ref_vol, cond=is_greater_than_zero, cond_args=None, exclusion=None, exclusion_args=None):
    """Returns coordinates [cor, sag, axial]

    Args:
        ref_vol (np.ndarray | tuple[np.ndarray]): 

    Returns:
        _type_: tuple[int]
    """
    if cond_args is None:
        cond_args = []
    
    if exclusion_args is None:
        exclusion_args = []
    
    if exclusion is None:
        def exclusion(_):
            return np.array([False, False])
        
    if isinstance(ref_vol, np.ndarray):
        ref_x = ref_y = ref_z = ref_vol
    else:
        ref_x, ref_y, ref_z = ref_vol

    check_slice_x = 0
    n_voxels_best = 0
    for i in range(0, ref_x.shape[0]):
        ref_slice_mask = ref_x[i, :, :]
        if (exclusion(ref_slice_mask, *exclusion_args)).any():
            continue
        n_voxels = np.sum(cond(ref_slice_mask, *cond_args))
        if  n_voxels > n_voxels_best:
            n_voxels_best = n_voxels
            check_slice_x = i


    check_slice_y = 0
    n_voxels_best = 0
    for i in range(0, ref_y.shape[1]):
        ref_slice_mask = ref_y[:, i, :]
        if (exclusion(ref_slice_mask, *exclusion_args)).any():
            continue
        n_voxels = np.sum(cond(ref_slice_mask, *cond_args))
        if  n_voxels > n_voxels_best:
            n_voxels_best = n_voxels
            check_slice_y = i


    check_slice_z = 0
    n_voxels_best = 0
    for i in range(0, ref_z.shape[2]):
        ref_slice_mask = ref_z[:, :, i]
        if (exclusion(ref_slice_mask, *exclusion_args)).any():
            continue
        n_voxels = np.sum(cond(ref_slice_mask, *cond_args))
        if  n_voxels > n_voxels_best:
            n_voxels_best = n_voxels
            check_slice_z = i

    return check_slice_x, check_slice_y, check_slice_z

#### Save basic sagittal view

In [46]:
# save_root = image_root / "sagittal_view_basic"
# if not save_root.exists():
#     os.makedirs(save_root)

# failed_subs = []
# for sub in all_subjects:
#     sub_root = subject_roots[sub]
#     # cp_seg = nib.load(sub_root/"choroid.nii.gz").get_fdata()
#     # vent_seg = nib.load(sub_root/"aseg-ventricles.nii.gz").get_fdata()
#     try:
#         fourth_vent_seg = nib.load(sub_root/"aseg-fourth_ventricle.nii.gz").get_fdata()
#         # third_vent_seg = nib.load(sub_root/"aseg-third_ventricle.nii.gz").get_fdata()

#         coords = get_ref_slice(fourth_vent_seg)
#     except Exception:
#         failed_subs.append(sub)
#         coords = (105, 120, 194)
#     coord_dict = {
#         "coronal": coords[0],
#         "sagittal": coords[1],
#         "axial": coords[2]
#     }
#     anat = sub_root / "t1.nii.gz"
#     viewport = "sagittal"

#     save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-{viewport}[{coord_dict[viewport]}].jpg"

#     cmd = [
#         screenshot_script,
#         anat,
#         viewport,
#         save_path,
#         str(coords[0]),
#         str(coords[1]),
#         str(coords[2])
#     ]
#     if not save_path.exists():
#         print(sub)
#         try:
#             result = subprocess.run(cmd, check=True, capture_output=True)
#         except subprocess.CalledProcessError:
#             failed_subs.append(sub)
#         if not save_path.exists():
#             failed_subs.append(sub)


In [47]:
# save_root = image_root / "hips_thomas"
# if not save_root.exists():
#     os.makedirs(save_root)

# failed_subs = []
# for sub in all_subjects:
#     sub_root = subject_roots[sub]
#     try:
#         fourth_vent_seg = nib.load(sub_root/"aseg-fourth_ventricle.nii.gz").get_fdata()
#         # third_vent_seg = nib.load(sub_root/"aseg-third_ventricle.nii.gz").get_fdata()

#         coords = get_ref_slice(fourth_vent_seg)
#     except Exception:
#         failed_subs.append(sub)
#         coords = (105, 120, 194)
#     coord_dict = {
#         "coronal": coords[0],
#         "sagittal": coords[1],
#         "axial": coords[2]
#     }
#     anat = sub_root / "t1.nii.gz"
#     viewport = "sagittal"

#     save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-{viewport}[{coord_dict[viewport]}].jpg"

#     cmd = [
#         screenshot_script,
#         anat,
#         viewport,
#         save_path,
#         str(coords[0]),
#         str(coords[1]),
#         str(coords[2])
#     ]
#     if not save_path.exists():
#         print(sub)
#         try:
#             result = subprocess.run(cmd, check=True, capture_output=True)
#         except subprocess.CalledProcessError:
#             failed_subs.append(sub)
#         if not save_path.exists():
#             failed_subs.append(sub)


In [48]:
failed_subs_prev = [1011, 1015, 1019, 1189, 1189, 1196, 1196, 1196, 1199]

In [49]:
# sub = 1019
# sub_root = subject_roots[sub]
# # cp_seg = nib.load(sub_root/"choroid.nii.gz").get_fdata()
# # vent_seg = nib.load(sub_root/"aseg-ventricles.nii.gz").get_fdata()
# try:
#     fourth_vent_seg = nib.load(sub_root/"aseg-fourth_ventricle.nii.gz").get_fdata()
#     # third_vent_seg = nib.load(sub_root/"aseg-third_ventricle.nii.gz").get_fdata()

#     coords = get_ref_slice(fourth_vent_seg)
# except Exception:
#     failed_subs.append(sub)
#     coords = (105, 120, 194)
# coord_dict = {
#     "coronal": coords[0],
#     "sagittal": coords[1],
#     "axial": coords[2]
# }
# anat = sub_root / "t1.nii.gz"
# viewport = "sagittal"

# save_path = image_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-{viewport}[{coord_dict[viewport]}].jpg"

# cmd = [
#     screenshot_script,
#     anat,
#     viewport,
#     save_path,
#     str(coords[0]),
#     str(coords[1]),
#     str(coords[2])
# ]
# print(sub)
# subprocess.run(cmd, check=True, capture_output=True)

### HIPS-THOMAS

#### Thalamus Largest

In [ ]:
failed_subs = []

cond = is_equal_to
cond_args = [thalamic_nuclei]


save_root = image_root / "hips_thomas_thalamus_right"
if not save_root.exists():
    os.makedirs(save_root)
   
for sub in tqdm(all_subjects):
   try:
      sub_root = subject_roots[sub]

      anat = sub_root / "t1.nii.gz"
      hipsthomas_l = sub_root / "thomasfull_L.nii.gz"
      hipsthomas_r = sub_root / "thomasfull_R.nii.gz"
      choroid = sub_root / "choroid.nii.gz"

      ref_vol = nib.load(hipsthomas_l).get_fdata()
      x, y, z = get_ref_slice(ref_vol, cond=cond, cond_args=cond_args)

      coord_dict = {
         "coronal": str(x),
         "sagittal": str(y),
         "axial": str(z)
      }
      anat = sub_root / "t1.nii.gz"

      viewport = "axial"

      save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-seg_hipsthomas-{viewport}[{coord_dict[viewport]}].png"

      if not save_path.exists():
         cmd = ["fsleyes", "render", "--outfile", f"\"{str(save_path)}\"", "--size", "1080", "1080", 
               "--scene", "ortho", "-xh" , "-yh", 
               "--voxelLoc", *coord_dict.values(),
               "--hideCursor", "--hideLabels",
               str(anat), "-ot", "volume",
               str(hipsthomas_l), "-ot", "label", "-l", "freesurfercolorlut", "-o",
               str(hipsthomas_r), "-ot", "label", "-l", "freesurfercolorlut", "-o",
               str(choroid), "-ot", "label", "-l", "freesurfercolorlut", "-o"]

         script_file = "script.bash"
         with open(script_file, 'w') as f:
            f.write("#!/bin/bash\n")
            f.write(" ".join(cmd))
         try:
            subprocess.run(["bash", script_file], check=True, capture_output=True)
         except subprocess.CalledProcessError as e:
            failed_subs.append(sub)
            print(e)
   except Exception:
      failed_subs.append(sub)

  0%|          | 0/575 [00:00<?, ?it/s]

Command '['bash', 'script.bash']' returned non-zero exit status 1.


### Lateral ventricles

In [51]:
failed_subs = []

check_name = "lateral_ventricles"
viewport = "axial"

save_root = image_root / "lateral_ventricles"
if not save_root.exists():
    os.makedirs(save_root)
   
for sub in tqdm(all_subjects):
   try:
      sub_root = subject_roots[sub]

      anat = sub_root / "t1.nii.gz"
      segmentation = sub_root / "aseg-lateral_ventricles.nii.gz"

      ref_vol = nib.load(segmentation).get_fdata()
      x, y, z = get_ref_slice(ref_vol)

      coord_dict = {
         "coronal": str(x),
         "sagittal": str(y),
         "axial": str(z)
      }
      anat = sub_root / "t1.nii.gz"


      save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-seg_{check_name}-{viewport}[{coord_dict[viewport]}].png"

      if not save_path.exists():
         cmd = ["fsleyes", "render", "--outfile", f"\"{str(save_path)}\"", "--size", "1080", "1080", 
               "--scene", "ortho", "-xh" , "-yh", 
               "--voxelLoc", *coord_dict.values(),
               "--hideCursor", "--hideLabels",
               str(anat), "-ot", "volume",
               str(segmentation), "-ot", "label", "-l", "freesurfercolorlut", "-o"]

         script_file = "script.bash"
         with open(script_file, 'w') as f:
            f.write("#!/bin/bash\n")
            f.write(" ".join(cmd))
         try:
            subprocess.run(["bash", script_file], check=True, capture_output=True)
         except subprocess.CalledProcessError as e:
            failed_subs.append(sub)
            print(e)
   except Exception as e:
      failed_subs.append(sub)
      print(e)

  0%|          | 0/575 [00:00<?, ?it/s]

No such file or no access: '/media/smbshare/srs-9/thalamus_project/data/sub1196-20161004/aseg-lateral_ventricles.nii.gz'


#### Choroid Largest Slice

In [52]:
failed_subs = []

check_name = "choroid"
ref_name = check_name
viewport = "axial"

save_root = image_root / "choroid"
if not save_root.exists():
    os.makedirs(save_root)
   
for sub in tqdm(all_subjects):
   try:
      sub_root = subject_roots[sub]

      anat = sub_root / "t1.nii.gz"
      segmentation = sub_root / f"{check_name}.nii.gz"
      ref_path = sub_root / f"{ref_name}.nii.gz"

      ref_vol = nib.load(ref_path).get_fdata()
         
      x, y, z = get_ref_slice(ref_vol)

      coord_dict = {
         "coronal": str(x),
         "sagittal": str(y),
         "axial": str(z)
      }
      anat = sub_root / "t1.nii.gz"


      save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-seg_{check_name}-{viewport}[{coord_dict[viewport]}].png"

      if not save_path.exists():
         cmd = ["fsleyes", "render", "--outfile", f"\"{str(save_path)}\"", "--size", "1080", "1080", 
               "--scene", "ortho", "-xh" , "-yh", 
               "--voxelLoc", *coord_dict.values(),
               "--hideCursor", "--hideLabels",
               str(anat), "-ot", "volume",
               str(segmentation), "-ot", "label", "-l", "freesurfercolorlut", "-o"]

         script_file = "script.bash"
         with open(script_file, 'w') as f:
            f.write("#!/bin/bash\n")
            f.write(" ".join(cmd))

         try:
            subprocess.run(["bash", script_file], check=True, capture_output=True)
         except subprocess.CalledProcessError as e:
            failed_subs.append(sub)
            print(e)
   except Exception as e:
      failed_subs.append(sub)
      print(e)

  0%|          | 0/575 [00:00<?, ?it/s]

No such file or no access: '/media/smbshare/srs-9/thalamus_project/data/sub1196-20161004/choroid.nii.gz'


#### Upper Choroid

In [53]:
failed_subs = []

cond = is_equal_to
cond_args = [(31)]
exclusion = is_equal_to
exclusion_args = [tuple(set(range(1, 100)) - set([27, 31]))]

check_name = "choroid"
ref_name = "thomasfull_L"
viewport = "axial"

save_root = image_root / "choroid_upper"
if not save_root.exists():
    os.makedirs(save_root)
   
for sub in tqdm(all_subjects):
   try:
      sub_root = subject_roots[sub]

      anat = sub_root / "t1.nii.gz"
      segmentation = sub_root / f"{check_name}.nii.gz"
      ref_path = sub_root / f"{ref_name}.nii.gz"

      ref_vol = nib.load(ref_path).get_fdata()
         
      _, _, z = get_ref_slice(ref_vol, cond=cond, cond_args=cond_args, exclusion=exclusion, exclusion_args=exclusion_args)
      if z == 0:
         _, _, z = get_ref_slice(ref_vol, cond=cond, cond_args=[(27)])

      x, y, _ = get_ref_slice(ref_vol)

      coord_dict = {
         "coronal": str(x),
         "sagittal": str(y),
         "axial": str(z)
      }
      anat = sub_root / "t1.nii.gz"


      save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-seg_{check_name}-{viewport}[{coord_dict[viewport]}].png"

      if not save_path.exists():
         cmd = ["fsleyes", "render", "--outfile", f"\"{str(save_path)}\"", "--size", "1080", "1080", 
               "--scene", "ortho", "-xh" , "-yh", 
               "--voxelLoc", *coord_dict.values(),
               "--hideCursor", "--hideLabels",
               str(anat), "-ot", "volume",
               str(segmentation), "-ot", "label", "-l", "freesurfercolorlut", "-o"]

         script_file = "script.bash"
         with open(script_file, 'w') as f:
            f.write("#!/bin/bash\n")
            f.write(" ".join(cmd))

         try:
            subprocess.run(["bash", script_file], check=True, capture_output=True)
         except subprocess.CalledProcessError as e:
            failed_subs.append(sub)
            print(e)
   except Exception as e:
      failed_subs.append(sub)
      print(e)

  0%|          | 0/575 [00:00<?, ?it/s]

Command '['bash', 'script.bash']' returned non-zero exit status 1.


#### Lower Choroid

In [54]:
failed_subs = []

cond = is_equal_to
cond_args = [(8)]

check_name = "choroid"
ref_name = "thomasfull_L"
viewport = "axial"

save_root = image_root / "choroid_lower"
if not save_root.exists():
    os.makedirs(save_root)
   
for sub in tqdm(all_subjects):
   try:
      sub_root = subject_roots[sub]

      anat = sub_root / "t1.nii.gz"
      segmentation = sub_root / f"{check_name}.nii.gz"
      ref_path = sub_root / f"{ref_name}.nii.gz"

      ref_vol = nib.load(ref_path).get_fdata()
         
      _, _, z = get_ref_slice(ref_vol, cond=cond, cond_args=cond_args)
      x, y, _ = get_ref_slice(ref_vol)

      coord_dict = {
         "coronal": str(x),
         "sagittal": str(y),
         "axial": str(z)
      }
      anat = sub_root / "t1.nii.gz"


      save_path = save_root / f"sub{sub}-anat_{utils.nifti_name(anat.name)}-seg_{check_name}-{viewport}[{coord_dict[viewport]}].png"

      if not save_path.exists():
         cmd = ["fsleyes", "render", "--outfile", f"\"{str(save_path)}\"", "--size", "1080", "1080", 
               "--scene", "ortho", "-xh" , "-yh", 
               "--voxelLoc", *coord_dict.values(),
               "--hideCursor", "--hideLabels",
               str(anat), "-ot", "volume",
               str(segmentation), "-ot", "label", "-l", "freesurfercolorlut", "-o"]

         script_file = "script.bash"
         with open(script_file, 'w') as f:
            f.write("#!/bin/bash\n")
            f.write(" ".join(cmd))

         try:
            subprocess.run(["bash", script_file], check=True, capture_output=True)
         except subprocess.CalledProcessError as e:
            failed_subs.append(sub)
            print(e)
   except Exception as e:
      failed_subs.append(sub)
      print(e)

  0%|          | 0/575 [00:00<?, ?it/s]

Command '['bash', 'script.bash']' returned non-zero exit status 1.
